In [81]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [76]:
sample_submission = pd.read_csv("./_data/sample_submission.csv")
stock_list = pd.read_csv("./_data/stock_list.csv")
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6)) #zfill 숫자만큼 자릿수 맞춰 0 채우기


In [61]:
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [77]:
stock_code = '005930' #삼성전자
stock_code ='035420'
predict_date = '20211129'  # 29일의 종가 예측
period = 365         # 1년간의 데이터를 사용

In [78]:
predict_date = pd.to_datetime(predict_date)
start_date = '20210104'
end_date = '20211105'

In [79]:
start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

In [65]:
start_weekday

0

In [66]:
max_weeknum

'44'

In [67]:
Business_days

,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08
...,...
215,2021-11-01
216,2021-11-02
217,2021-11-03
218,2021-11-04


In [80]:
stock = fdr.DataReader(stock_code, start=start_date, end=end_date)[["Close"]].reset_index()

In [69]:
stock

,Date,Close
0,2021-01-04,293000
1,2021-01-05,292500
2,2021-01-06,290000
3,2021-01-07,289500
4,2021-01-08,312000
...,...,...
204,2021-11-01,408500
205,2021-11-02,410000
206,2021-11-03,402500
207,2021-11-04,411500


In [70]:
stock = pd.merge(Business_days, stock, how='outer')
stock['weekday'] = stock.Date.apply(lambda x : x.weekday())
stock['weeknum'] = stock.Date.apply(lambda x : x.strftime('%V'))
stock.Close = stock.Close.ffill()
stock = pd.pivot_table(data = stock, values = 'Close', columns = 'weekday', index = 'weeknum')
stock.head()


weekday,0,1,2,3,4
weeknum,,,,,
01,293000.0,292500.0,290000.0,289500.0,312000.0
02,309000.0,304000.0,314000.0,318000.0,306000.0
03,300500.0,308000.0,308000.0,322500.0,343500.0
04,349000.0,342000.0,341500.0,355000.0,343000.0
05,348000.0,362500.0,370500.0,358000.0,362500.0


In [71]:

from sklearn.linear_model import LinearRegression
model = LinearRegression()
x = stock.iloc[0:-2].to_numpy()
x.shape

(42, 5)

In [36]:
y = sample.iloc[1:-1].to_numpy()
y_0 = y[:,0]
y_1 = y[:,1]
y_2 = y[:,2]
y_3 = y[:,3]
y_4 = y[:,4]

y_values = [y_0, y_1, y_2, y_3, y_4]
x_public = sample.iloc[-2].to_numpy()

In [82]:
predictions = []
for y_value in y_values :
    X_train, X_test, y_train, y_test = train_test_split(x, y_value, random_state=11, test_size=0.3)
    model.fit(X_train,y_train)
    score = model.score(X_test, y_test)
    prediction = model.predict(np.expand_dims(x_public,0))
    print(f"{prediction}")
    print(f"테스트 정확도:{score}")


    predictions.append(prediction[0])
#predictions

[108203.55088251]
테스트 정확도:-0.4201362293799922
[109184.84791057]
테스트 정확도:-0.3262521542971113
[107582.48344749]
테스트 정확도:-0.28389371838698674
[106521.74505204]
테스트 정확도:-0.11326359591531898
[105411.87467592]
테스트 정확도:0.05996633794318951


In [84]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                           LogisticRegression(random_state=1))
pipe_lr.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])